In [ ]:
%%capture
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
log = logging.getLogger()
log.setLevel(logging.DEBUG)

import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
from hyperoptim.results import load_data_frame, remove_columns_with_same_value, dataset_averaged_frame
from hyperoptim.results import mean_identical_exps
import pandas as pd
from hyperoptim.results import remove_columns_with_same_value, remove_indices_with_same_value

import json
from collections import OrderedDict

from copy import deepcopy

In [ ]:

df = load_data_frame('data/models/pytorch/auto-diag/less-minutes/')
df = df[df.finished == 1]
df = df.fillna('-')
df = df.drop('seed', axis=1)
df = df.drop(['save_predictions', 'save_folder'], axis=1)
df.runtime = pd.to_timedelta(np.round(df.runtime), unit='s')
df = remove_columns_with_same_value(df)
print(len(df))
df = df.rename(columns=dict(shrink_the_spikes='shrink',
                            channel_standardize='chan_std',
                            low_cut_hz='low_hz',
                            max_abs_val='max_val',
                            high_cut_hz='high_hz',
                            moving_demean='m_mean',
                            moving_standardize='m_std')).drop(['train_loss', 'valid_loss',
                                                       'test_loss', 'valid_misclass',
                                                       ], axis=1)
#df = df.drop(['save_predictions', 'save_crop_predictions',], axis=1)
df.test_misclass = df.test_misclass * 100

df['test_accuracy'] = 100 - df.test_misclass
print(len(df))
df = df.sort_values(by=['test_recording_mins','duration_recording_mins'])
df

In [ ]:
minute_steps, all_trial_accs, all_crop_accs = np.load(
    'data/models/pytorch/auto-diag/less-minutes/12/reduced_time_accuracies.npy')
minute_steps = minute_steps[1:]
all_trial_accs = all_trial_accs[1:]
all_crop_accs = all_crop_accs[1:]

In [ ]:
plt.figure(figsize=(8,2))
df_train_test_reduced = df[df.test_recording_mins == '-']
plt.plot(np.array(df_train_test_reduced.duration_recording_mins), np.array(df_train_test_reduced.test_accuracy),
        color=seaborn.color_palette()[0], label='Reduced train and test')
plt.plot(np.array(df_train_test_reduced.duration_recording_mins), 
         100 - 100*np.array(df_train_test_reduced.test_sample_misclass),
        color=seaborn.color_palette()[0], ls=':')
df_train_reduced = df[df.test_recording_mins == 20]
plt.plot(np.array(df_train_reduced.duration_recording_mins), np.array(df_train_reduced.test_accuracy),
        color=seaborn.color_palette()[1], label='Reduced train only')
plt.plot(np.array(df_train_reduced.duration_recording_mins), 
         100 - 100*np.array(df_train_reduced.test_sample_misclass),
        color=seaborn.color_palette()[1], ls=':')
plt.plot(minute_steps, all_trial_accs * 100,
        color=seaborn.color_palette()[2], label='Reduced test only')
plt.plot(minute_steps, all_crop_accs * 100,
        color=seaborn.color_palette()[2], ls=':')

plt.xscale('symlog',basex=2)
plt.xticks(np.array(df_train_test_reduced.duration_recording_mins), fontsize=12)
plt.yticks([80,82,84,86], fontsize=12)
plt.gca().get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
plt.xlabel("Time [minutes]", fontsize=12)
plt.ylabel("Test Accuracy [%]", fontsize=12)
#plt.legend(bbox_to_anchor=(1,1,0,0));
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
from matplotlib.lines import Line2D

handles.append(Line2D([0,1], [0,0], color='black', 
    markersize=9, linewidth=1.5))
handles.append(Line2D([0,1], [0,0], color='black', 
    markersize=9, linewidth=1.5, ls=":"))
labels.append('Trialwise')
labels.append('Cropwise')

ax.legend(handles, labels, 
          bbox_to_anchor=(1,1,0,0),
         fontsize=12)
plt.savefig('/home/schirrmr/code/auto-diagnosis/Time_Plot.pdf', bbox_inches='tight', dpi=600, pad_inches=0)

In [ ]:
# Created in https://10.5.166.71:9999/notebooks/code/auto-diagnosis/notebooks/plots/Prediction_Less_Test_Time.ipynb
correct_timecourse, pred_correct = np.load('data/models/pytorch/auto-diag/less-minutes/12/correct_timecourse.npy')

from rtsutil.stats import running_mean
plt.figure(figsize=(8,2))
n_smooth_samples = 60000
smoothed_timecourse = running_mean(correct_timecourse,n_smooth_samples,)
plt.plot((n_smooth_samples / 2 + 600 + np.arange(len(smoothed_timecourse))) / 100.0, smoothed_timecourse)


plt.xlabel('Time [sec]', fontsize=14)
plt.ylabel('Accuracy [%]', fontsize=14)
#plt.title("5-minute smoothing", fontsize=16)
plt.xlim(200,1000)
plt.savefig('/home/schirrmr/code/auto-diagnosis/Time_Crop_Pred_Plot.pdf', bbox_inches='tight', dpi=600, pad_inches=0)